<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/ExothermicPFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

In [51]:
N=15

q = 100. #L/min
Cain = 1. #mol/L
Tin = 350. #K
V = 100./N #L
rho = 1e3 #g/L
Cp = 0.239 #J/(g K)
negHr = 5e4 #J/mol
ER = 8750. #K
k0 = 7.2e10 #1/min
UA = 5e4/N #J/(min K)
Tc = 290. #K

Ca0 = np.full(N,0.5) #mol/L
T0 = np.full(N,350.) #K
tend= 3. #min

In [52]:
def rhs(t, vec):
    Ca= vec[:N]
    T = vec[N:]
    k = k0*np.exp(-ER/T)

    #1st CSTR
    dCa0 = q*(Cain - Ca[0])/V - k[0]*Ca[0]
    dT0 = q*(Tin - T[0])/V - UA/(rho*V*Cp) * (T[0]-Tc) + negHr*k[0]*Ca[0]/(rho*Cp)

    #remaining CSTR
    dCa = q*(Ca[:-1] - Ca[1:])/V - k[1:]*Ca[1:]
    dT = q*(T[:-1] - T[1:])/V - UA/(rho*V*Cp) * (T[1:]-Tc) + negHr*k[1:]*Ca[1:]/(rho*Cp)

    return  np.concatenate([np.r_[dCa0, dCa],  np.r_[dT0, dT]])

In [53]:
res=solve_ivp(rhs, (0,tend), np.concatenate([Ca0,T0]), method='Radau', dense_output=True)

In [54]:
time_plot = np.linspace(0,tend,500)
Ca, T= res.sol(time_plot).reshape(2,N,-1)

fig = make_subplots(rows=1, cols=1)

for i in range(N):
    fig.add_scatter(x=time_plot, y=Ca[i,:], name=f'Ca{i}',row=1,col=1)

fig.update_layout(width=500, height=500, template='plotly_dark')

In [56]:
fig2 = make_subplots(rows=1, cols=1)

for i in range(N):
    fig2.add_scatter(x=time_plot, y=T[i,:], name=f'T{i}', row=1, col=1)

fig2.update_layout(width=500, height=500, template='plotly_dark')

